Set some basic config parameters:

In [ ]:
embed_size = 100 # how big is each word vector
max_features = 10000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use
# consider increase to 200 to 300 

Read in our data and replace missing values:

In [ ]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)
# fasttext preprocessing
train[COMMENT] = train[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace(':',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.replace('\n','NEWLINE ').str.strip().str.replace("[^\s\d\w)(;:\-+_?!\]\[,*,'\"]+","")
test[COMMENT] = test[COMMENT].str.lower().str.replace('https?:\/\/[^\s]*','').str.replace("’","'").str.replace("''",' ').str.replace("'"," ' ").str.replace('“','"').str.replace('”','"').str.replace(':',' : ').str.replace('.',' . ').str.replace(',',' , ').str.replace('[',' [ ').str.replace(']',' ] ').str.replace('(',' ( ').str.replace(')',' ) ').str.replace('!',' ! ').str.replace('?',' ? ').str.replace(';',' ').str.replace(':',' ').str.replace('-',' - ').str.replace('=', ' ').str.replace('=', ' ').str.replace('*', ' ').str.replace('|', ' ').str.replace('«', ' ').str.replace('\d', ' ').str.replace('\n', ' ').str.replace('\s\s+',' ').str.replace('\n','NEWLINE ').str.strip().str.replace("[^\s\d\w)(;:\-+_?!\]\[,*,'\"]+","")

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [ ]:
# TODO test average, max, min comments words
test_train = train.copy()
test_train['tokens'] = test_train[COMMENT].str.split()

In [ ]:
test_train['tokens_len'] = test_train['tokens'].apply(lambda x: len(x))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
print('mean:',test_train['tokens_len'].mean(), 'min:',test_train['tokens_len'].min(), 'max:',test_train['tokens_len'].max())

In [ ]:
test_train['tokens_len'].sample(n=3000).mean()

In [ ]:
test_train['tokens_len'].sample(n=3000).plot(kind='density')

Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
# TODO filters=''
# Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Read the wiki user comments word vectors (space delimited strings) into a dictionary from word->vector.